## Overview

Here we demo the basic operation of the `noisy load profiles` package. recall that the goal is to add realistic noise to a given load profile.

We do this by choosing various `Perturbation` which are sources of noise, such as `MultiplicativeGaussianNoise` or `'PercentualDeadBand'`. 

These `Perturbations` are added to a `Pipeline` object. This Pipeline is than applied to a given matrix of load profiles. 

NOTE: we always assume rows = time and columns = measurement devices

In [23]:
import noisy_load_profiles as nolp
from noisy_load_profiles import Pipeline
from noisy_load_profiles.perturbations import MultiplicativeGaussianNoise, ZeroMeasurements
import numpy as np

In [24]:
print("All available perturbations:")
available_perturbations = dir(nolp.perturbations)
available_perturbations = [p for p in available_perturbations if not p.startswith('_')]  # remove dunder methods such as __builtins__
available_perturbations = [p for p in available_perturbations if p not in ['measurement', 'random', 'systematic']]  ## filter out non-perturbation classes

for p in available_perturbations:
    print(f"  {p}")

All available perturbations:
  AdditiveOUNoise
  ConstantRandomPercentualBias
  ConstantRandomPercentualScaling
  DiscreteTimeShift
  MultiplicativeGaussianNoise
  PercentualDeadBand
  ZeroMeasurements


## Example of a simple Pipeline

In [5]:
# built a profiles
timesteps = 10
n_profiles = 2
profiles = np.ones((timesteps, n_profiles)) # 2 profiles with 10 timesteps each; example


# initialize a perturbation
gaussian_noise = MultiplicativeGaussianNoise(mean=0.0, std=0.01, seed=42)

# add a perturbation to a pipeline
pipeline = Pipeline([gaussian_noise])


# apply the perturbation to the profiles
perturbed_profiles = pipeline.apply(profiles)


# print the first row of both profiles
print("Original profiles:")
print(profiles[0, :])

print("Perturbed profiles:")
print(perturbed_profiles[0, :])

Original profiles:
[1. 1.]
Perturbed profiles:
[1.00496714 0.99861736]


In [7]:
# you can also find the transformations of each perturbation
print("Transformations:")
pipeline.get_transformations()

Transformations:


{MultiplicativeGaussianNoise(mean=0.0, std=0.01): {'noise_samples': array([[ 0.00496714, -0.00138264],
         [ 0.00647689,  0.0152303 ],
         [-0.00234153, -0.00234137],
         [ 0.01579213,  0.00767435],
         [-0.00469474,  0.0054256 ],
         [-0.00463418, -0.0046573 ],
         [ 0.00241962, -0.0191328 ],
         [-0.01724918, -0.00562288],
         [-0.01012831,  0.00314247],
         [-0.00908024, -0.01412304]]),
  'mean': 0.0,
  'std': 0.01,
  'shape': (10, 2)}}

## Longer pipeline

In [17]:
help(nolp.perturbations.ZeroMeasurements)

Help on class ZeroMeasurements in module noisy_load_profiles.perturbations.measurement:

class ZeroMeasurements(noisy_load_profiles.base.Perturbation)
 |  ZeroMeasurements(f_zero: Optional[float] = None, k_max: Optional[int] = None, tail_eps: float = 0.1, alpha: Optional[float] = None, beta: Optional[float] = None, per_column_independent: bool = True, seed: Optional[int] = None, transformation: Optional[Dict[str, Any]] = None, track_input_profiles: bool = False)
 |
 |  Put Simply, this sometimes sets a measurements to zero, simulating an outage or a measurement error.
 |
 |  This is done using a 2-state Markov Model
 |
 |  States:
 |      0 = normal
 |      1 = zero/outage
 |
 |  Parameters
 |  ----------
 |  f_zero : float, optional
 |      Target long-run fraction of zeros, in (0, 1). Used to derive (alpha, beta)
 |      together with (k_max, tail_eps).
 |  k_max : int, optional
 |      "Likely maximum" run length. Together with tail_eps, sets beta via
 |      P(L > k_max) = tail_eps

In [19]:
# built a profiles
timesteps = 10
n_profiles = 2
profiles = np.ones((timesteps, n_profiles)) # 2 profiles with 10 timesteps each; example


# initialize a perturbation
ou_noise = nolp.perturbations.AdditiveOUNoise(seed=42)
deadband = nolp.perturbations.PercentualDeadBand(seed=42)
zeros = nolp.perturbations.ZeroMeasurements(f_zero=0.2, k_max=3, seed=42)

# make the pipeline
pipeline = Pipeline([ou_noise, deadband, zeros])

# apply the perturbation to the profiles
perturbed_profiles = pipeline.apply(profiles)

In [20]:
perturbed_profiles

array([[1.        , 0.        ],
       [1.02483571, 0.        ],
       [1.04480228, 0.        ],
       [1.01069347, 1.02464059],
       [1.08430738, 1.05069203],
       [1.01867997, 1.05069203],
       [0.        , 1.00295052],
       [0.        , 0.90581125],
       [0.        , 0.92479125],
       [0.        , 0.97810799]])